# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [5]:
!pip install langchain python-dotenv langchain-community langchain-groq google-generativeai

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

dotenv.load_dotenv()

True

In [30]:
loader = TextLoader('data.txt')
doc = loader.load()
doc

[Document(metadata={'source': 'data.txt'}, page_content="###Question###\nWho you are?,\n###Answer###\nYou are a chatbot designed for ACME Technologies Ltd to answer HR-related questions. Acme Technologies Ltd, Org, started as a modest real estate developer by a group of retired military officers and engineers. Later it entered into the construction of industrial and commercial complexes with a different name called Acme Constructions. By following basic business ethics, like, timely delivery, quality of work, and stunning architecture, Acme Technologies Ltd came into the limelight. \\n\\nTo provide its own needs and the needs of the clients it started a business in the service sector with a separate company called Acme Services. \\n\\nOf late, it also extended its business in the field of information technology. Its venture in IT is named Acme AI. Itâ€™s mainly doing annotation now but in the future, it plans to enter the other fields of IT to be a part of smart Bangladesh.,\n\n###Ques

In [31]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(doc)

In [34]:
texts[0]

Document(metadata={'source': 'data.txt'}, page_content='###Question###\nWho you are?,\n###Answer###\nYou are a chatbot designed for ACME Technologies Ltd to answer HR-related questions. Acme Technologies Ltd, Org, started as a modest real estate developer by a group of retired military officers and engineers. Later it entered into the construction of industrial and commercial complexes with a different name called Acme Constructions. By following basic business ethics, like, timely delivery, quality of work, and stunning architecture, Acme Technologies Ltd came into the limelight. \\n\\nTo provide its own needs and the needs of the clients it started a business in the service sector with a separate company called Acme Services. \\n\\nOf late, it also extended its business in the field of information technology. Its venture in IT is named Acme AI. Itâ€™s mainly doing annotation now but in the future, it plans to enter the other fields of IT to be a part of smart Bangladesh.,')

In [5]:
import pandas as pd

data = pd.read_csv('HR_DOC_Sheet1.csv')
data = data[:99]


In [6]:
len(data)

99

In [7]:
import chromadb
import uuid
import pandas as pd

# Initialize the ChromaDB client and collection
client = chromadb.PersistentClient('vectordb')
collection = client.get_or_create_collection(name="portfolio")

batch_size = 25  # Adjust batch size based on performance testing
count = 0
errors = 0
failed_rows = []

# Check if the collection is empty before adding new documents
if not collection.count():
    batch_questions = []
    batch_answers = []
    batch_ids = []
    
    for index, row in data.iterrows():
        question = row['Question']
        answer = row['Answer']
        
        # Check for missing values and skip if either is missing
        if pd.isna(question) or pd.isna(answer):
            errors += 1
            print(f"Skipping row {index}: Missing 'Question' or 'Answer'.")
            continue
        
        try:
            # Collect documents in batches
            batch_questions.append(question)
            batch_answers.append({"Answer": answer})
            batch_ids.append(str(uuid.uuid4()))
            
            # Add the batch when the batch size is reached
            if len(batch_questions) == batch_size:
                collection.add(documents=batch_questions, metadatas=batch_answers, ids=batch_ids)
                count += len(batch_questions)
                batch_questions.clear()
                batch_answers.clear()
                batch_ids.clear()
        except Exception as e:
            errors += 1
            failed_rows.append(index)
            print(f"Error on row {index}: {e}")
    
    # Add remaining documents in the last batch if any
    if batch_questions:
        try:
            collection.add(documents=batch_questions, metadatas=batch_answers, ids=batch_ids)
            count += len(batch_questions)
        except Exception as e:
            errors += 1
            failed_rows.extend(range(len(batch_questions)))
            print(f"Error on remaining rows: {e}")
    

In [8]:
    print(f"Total documents added: {collection.count()}")
    print(f"Total errors: {errors}")
    print(f"Failed rows: {failed_rows}")

Total documents added: 99
Total errors: 0
Failed rows: []


In [14]:
user_question = 'tell me about yourself'
links = collection.query(query_texts= user_question, n_results=1).get('metadatas')
links

[[{'Answer': 'Performance is measured by the collection of cash incentives within the next two quarters and 15 days.'}]]

In [15]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model="llama-3.1-8b-instant",
)

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    ### QUESTION FROM USER
    {user_question}
    ### ANSWER FROM VECTOR DATABASE CREATED BY CHROMADB
    {answer}
    ### INSTRUCTIONS
    Your job is to connect user question and answer and no need to repeat your question. for large response you can use points rather than heavy answer (NO PREAMBLE) 
    """    
)


In [17]:
chain = prompt_template | llm
res = chain.invoke(input={'user_question': user_question, 'answer': links[0][0]['Answer']})
res.content

'* I am an AI assistant trained on a vector database created by ChromaDB.\n* My primary function is to provide relevant and accurate information to users.\n* I can process and analyze large amounts of data to generate responses.\n* My performance is measured by the effectiveness of my responses in meeting user needs and expectations.\n* In this case, my performance is being evaluated based on the collection of cash incentives within a specific timeframe (next two quarters and 15 days).\n* I will strive to provide accurate and helpful responses to your questions and engage in productive conversations.'